In [ ]:
# !pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [1]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE
TESTSIZE = 200

In [2]:
#CHANGE THE FOLLOWING
EXPERT = "javascript" #python, cpp, java, javascript, etc.
MODELTYPE = "FINETUNED" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"
loramodel = f"mistral-7b-{EXPERT}-LORA-4bit"
lorarepo = f'santoshsto/{loramodel}'
PARTITION = "Javascript-program-level" #java-program-level etc.

In [3]:
from huggingface_hub import login
login(token="")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [4]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [5]:
from datasets import load_dataset
data = load_dataset(DATASET, PARTITION)

In [6]:
next(iter(data["test"]))

{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array',
 'code': 'function minSum ( A , N ) { let mp = new Map ( ) ; let sum = 0 ; for ( let i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; mp [ A [ i ] ] ++ ; if ( mp . has ( A [ i ] ) ) { mp . set ( A [ i ] , mp . get ( A [ i ] ) + 1 ) } else { mp . set ( A [ i ] , 1 ) } } let minSum = Number . MAX_SAFE_INTEGER ; for ( let it of mp ) { minSum = Math . min ( minSum , sum - ( it [ 0 ] * it [ 1 ] ) ) ; } return minSum ; } let arr = [ 4 , 5 , 6 , 6 ] ; let N = arr . length document . write ( minSum ( arr , N ) + " " ) ;'}

In [7]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]

In [8]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 8590
})
Dataset({
    features: ['text', 'code'],
    num_rows: 475
})
Dataset({
    features: ['text', 'code'],
    num_rows: 886
})


In [9]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b_eval = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
##########################################

################## 8bit ##################
bb_config_8b_eval = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config_eval(quantization):
    if quantization == "8bit":
        return bb_config_8b_eval
    else:
        return bb_config_4b_eval

In [10]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

if QUANTIZATION == "none":
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    ).to("cuda")
else:
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config_eval(QUANTIZATION),
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:49<00:00, 54.88s/it]


In [11]:
print(f'Base Model: {model_id}')
print(f'Adapeter: {lorarepo}')
model = PeftModel.from_pretrained(base_model, lorarepo)
#model = model_to_merge.merge_and_unload()

Base Model: mistralai/Mistral-7B-v0.1
Adapeter: santoshsto/mistral-7b-javascript-LORA-4bit


In [12]:
from torch.nn.utils import parameters_to_vector

total_params = parameters_to_vector(model.parameters()).numel()
total_params

3922153472

In [13]:
eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \\n code:</s>"""

In [14]:
# eval_prompt = f"""<s>
# For a given integer n, print out all its prime factors one on each line. 
# n = 30
# [INST]
# """
model.eval()
with torch.no_grad():
  input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)

  print(f"Prompt:\n{eval_prompt}\n")
  print(f"\nGenerated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(eval_prompt):]}")
  print(f'''\nGround truth:{next(iter(test_dataset))}''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Prompt:
<s>[INST] Write a javascript program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \n code:</s>


Generated response:
] \n function minSum ( arr , n ) { var freq = new Map ( ) ; for ( var i = 0 ; i < n ; i ++ ) { if ( freq . has ( arr [ i ] ) ) { freq . set ( arr [ i ] , freq . get ( arr [ i ] ) + 1 ) ; } else { freq . set ( arr [ i ] , 1 ) ; } } var sum = 0 ; for ( var i = 0 ; i < n ; i ++ ) { sum += arr [ i ] ; } var minSum = Number . MAX_VALUE ; for ( var [ key , value ] of freq . entries ( ) ) { minSum = Math . min ( minSum , sum - key * value ) ; } return minSum ; } var arr = [ 1 , 2 , 3 , 4 , 5 , 4 , 3 , 2 , 1 ] ; var N = arr

## Evaluation

In [15]:
import re
def extract_codeblock(response, prompt_length):
  delimiter = "code:"
 # Regular expression pattern to extract code block
  pattern = fr'{re.escape(delimiter)}(.*)'

  # Extract code block using regex
  match = re.search(pattern, response, re.DOTALL)

  if match:
      code_block = match.group(1)
      return code_block.strip()
  else:
      print("No code block found")
      return prediction[len(eval_prompt):]

In [16]:
from tqdm import tqdm
predictions = []
ground_truths = []
count = 0
pbar = tqdm(total=min(0, TESTSIZE), desc="Processing test dataset")

model.eval()
with torch.no_grad():
  for test_prompt in test_dataset:
    if count == TESTSIZE:
      break
    eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt['text']} [/INST] \\n </s>"""
    ground_truths.append(test_prompt['code'])
    input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)
    prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    predictions.append(prediction[len(eval_prompt):])
    count += 1
    pbar.update(1)

pbar.close()

Processing test dataset: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 1it [00:28, 28.35s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 2it [00:57, 28.70s/it]The attention mask and 

In [17]:
import pandas as pd

# Create a DataFrame with a single column named "value"
predictions_df = pd.DataFrame({"value": predictions})

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [18]:
predictions_df.head()

,value
0,"] \n function minSum ( arr , N ) { var freq = ..."
1,"let MAX = 1000000 ; function maxDiff ( arr , ..."
2,] \n function findSize ( N ) { if ( N == 1 ) r...
3,] \n function isPrime ( n ) { if ( n <= 1 ) re...
4,"let ans = 0 ; function gcd ( a , b ) { if ( b..."


In [19]:
references_df = pd.DataFrame({"value": ground_truths})
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu

In [20]:
from codebleu import calc_codebleu
results = []
for idx, reference in enumerate(ground_truths):
  prediction = predictions[idx]
  result = calc_codebleu([reference], [prediction], lang=EXPERT, weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
  results.append(result)
print(result)
df = pd.DataFrame(results)
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'
df.to_csv(score_file)
# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.49862429068250136, 'ngram_match_score': 0.3423895329498492, 'weighted_ngram_match_score': 0.503922481595008, 'syntax_match_score': 0.5584415584415584, 'dataflow_match_score': 0.5897435897435898}


In [21]:
df['codebleu'].mean()

0.41816533599259825

In [22]:
report_df = pd.concat([references_df, predictions_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file)

In [23]:
# from google.colab import files
# files.download(score_file)
# files.download(predictions_output_file)
# files.download(references_output_file)
# files.download(report_file)